In [2]:
import os
import logging
import pandas as pd
import mysql.connector

In [3]:
MYSQL_USERNAME=os.environ['MYSQL_USERNAME']
MYSQL_PASSWORD=os.environ['MYSQL_PASSWORD']
MYSQL_DATABASE=os.environ['MYSQL_DATABASE']
MYSQL_HOST=os.environ['MYSQL_HOST']

try:
    # Establish connection
    conn = mysql.connector.connect(
        host=MYSQL_HOST,
        user=MYSQL_USERNAME,
        password=MYSQL_PASSWORD,
        database=MYSQL_DATABASE
    )

    if conn.is_connected():
        print("Connected to MySQL successfully!")

except mysql.connector.Error as e:
    print("Error:", e)

Connected to MySQL successfully!


In [4]:
# Create a cursor object
cursor = conn.cursor()

In [5]:
def extract(cursor, query):
    # Execute a query
    cursor.execute(query)
    data = cursor.fetchall()
    
    # Get column names (metadata)
    columns = [desc[0] for desc in cursor.description]

    # Convert the data into a pandas DataFrame
    df = pd.DataFrame(data, columns=columns)
    
    return df

In [6]:
query="SELECT * FROM Employee;"
employees_df = extract(cursor, query)

In [7]:
employees_df

,EmployeeID,EmployeeName,NationalID,JobTitle,Gender,HireDate,Salary
0,1,John Doe,NID001,Store Manager,Male,2020-01-15,60000.0
1,2,Jane Smith,NID002,Sales Associate,Female,2021-03-20,35000.0
2,3,Emily Johnson,NID003,Cashier,Female,2022-05-01,30000.0
3,4,Michael Brown,NID004,Stock Clerk,Male,2022-08-30,28000.0
4,5,Sarah Wilson,NID005,Sales Associate,Female,2021-11-10,35000.0
5,6,David Garcia,NID006,Store Manager,Male,2019-06-01,60000.0
6,7,Laura Martinez,NID007,Cashier,Female,2022-12-20,30000.0
7,8,Tony Scott,NID008,Stock Clerk,Male,2023-01-15,28000.0
8,9,Linda Lee,NID009,Sales Associate,Female,2020-08-25,35000.0
9,10,George Harris,NID010,Warehouse Supervisor,Male,2018-04-12,45000.0


In [8]:
# Close the connection
cursor.close()
conn.close()
print("Connection closed.")

Connection closed.


In [9]:
import psycopg2
from psycopg2 import sql

In [10]:
PG_USERNAME=os.environ['PG_USERNAME']
PG_PASSWORD=os.environ['PG_PASSWORD']
PG_DATABASE=os.environ['PG_DATABASE']
PG_HOST=os.environ['PG_HOST']

print(PG_HOST, PG_USERNAME, PG_PASSWORD, PG_DATABASE)

172.18.0.4 biruni P@ssw0rd walmart_dwh


In [11]:
# Database connection parameters
db_config = {
    'dbname': PG_DATABASE,
    'user': PG_USERNAME,
    'password': PG_PASSWORD,
    'host': PG_HOST,
    'port': 5432
}


In [15]:
conn = psycopg2.connect(
            dbname=PG_DATABASE,
            user=PG_USERNAME,
            password=PG_PASSWORD,
            host=PG_HOST,
            port=5432
        )

In [16]:
# Connect to the database
try:
    conn = psycopg2.connect(**db_config)
    print("Connected to the database!")
except Exception as e:
    print(f"Unable to connect to the database: {e}")


Connected to the database!


In [17]:
# Create a cursor object to interact with the database
cursor = conn.cursor()

In [18]:
query = sql.SQL(f"""
    INSERT INTO dimemployee (employeebk, employeename, nationalid, jobtitle, gender, hiredate, salary)
    VALUES (%s, %s, %s, %s, %s, %s, %s);
""")

In [19]:
def load(cursor, query, data):
    for i, row in data.iterrows():

        try:
            cursor.execute(query, tuple(row))
        except Exception as e:
            conn.rollback()  # Rollback in case of error
            print(f"Error loading employee: {e}")

    cursor.execute('commit')
    print("Data Loaded successfully!")

In [20]:
load(cursor, query, employees_df)

Data Loaded successfully!


In [12]:
# Close the cursor and connection
cursor.close()
conn.close()
print("Connection closed.")

Connection closed.
